In [ ]:
#using Pkg;
#Pkg.add(url="https://github.com/FerreolS/OptimPackNextGen.jl",rev="verbose")

In [ ]:
include("../src/SphereIFSCalib.jl")

using TwoDimensional, Zygote, StatsBase, Plots, OptimPackNextGen, FITSIO, DelimitedFiles, Tables, DataFrames, Statistics
#using HypothesisTests #contient ChisqTest

plotly()

### Constantes

In [ ]:
# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
#const λ4 = 1545.10e-9  # laser 4  
λlaser = [λ1,λ2,λ3]
λ0 = mean(λlaser);# reference
wavelengthrange = LinRange(850e-9,1600e-9,50); # coarse wavelength range of the instrument

## Fit global sur l'ensemble des lenslets:

On definit les parametres qui servent a creer le modele:
a l'amplitude maximale des spots gaussien,
fwhm la largeur a mi hauteur des spots,
cx les coefficients polynomiaux le long de l'axe x,
cy les coefficients polynomiaux le long de l'axe y

### Exemple

In [ ]:
#Bbox of the lenslet: box autour d'un set de 3 gaussiennes
bbox = BoundingBox(xmin=1012, ymin=1048, xmax=1018, ymax=1092);
a0 = [300.0 , 400.0, 500.0];
fwhm0 = [2.0, 2.0 , 2.0];
#affichage d'un exemple de trois gaussiennes avec la bbox sur les donnees reelles:
exemple = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_wave_corrected.fits")
data= read(exemple[1]);
exempleData = data[1013:1017 , 1050:1090]
heatmap(exempleData, title = "Lenslet correspondant a la bbox initiale (exemple)", aspect_ratio = 1)

On peut deduire de cette image la taille de la bounding box d'une lenslet: hauteur 4 pixels, largeur 40 pixels.
Il manque les coeff C0 pour pouvoir fiter les gaussiennes, qu'on recupere dans des fichiers txt

### 1 lecture des fichiers txt

On veut recuperer en sortie de fonction la premiere colonne (c0, coef a l'ordre 0, soit le centre de la lenslet).

x(λ) = cx0 + cx1*((λ-λ0)/λ0) + cx2*((λ-λ0)/λ0)²

y(λ) = cy0 + cy1*((λ-λ0)/λ0) + cy2*((λ-λ0)/λ0)²

On donne ici λ en metre, dans le code Matlab λ est en micronmetre. Il faut donc normaliser et convertir les coefficients c1 et c2. 

In [ ]:
coeffx = readdlm("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/coef_pol_x.txt", header = false)
cx0 = coeffx[:,1];
cx1 = coeffx[:,2]*λ0*1e6;
cx2 = coeffx[:,3]*λ0^2*1e12;

coeffy = readdlm("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/coef_pol_y.txt", header = false)
cy0 = coeffy[:,1];
cy1 = coeffy[:,2]*λ0*1e6;
cy2 = coeffy[:,3]*λ0^2*1e12;

### Histogrammes

In [ ]:
h1 = histogram(cx1, title = "cx1");
h2 = histogram(cx2, title = "cx2");
plot(h1, h2, legend = false)

In [ ]:
h3 = histogram(cy1, title = "cy1");
h4 = histogram(cy2, title = "cy2");
plot(h3, h4, legend = false)

### 2 ouvrir et trouver positions dans l'image brute

1/ IFS_sim_wave.fits: on ouvre le modele, qui a peu de BP, donc les fit des lenslets seront bons. f5 = FITS("/home/user/Pictures/IFS_sim_wave.fits")

2/ IFS_calib_wave_corrected: "vrai" fichier de depart (a comparer avec 1/ eventuellement)

In [ ]:
laserData = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_wave_corrected.fits", "r");

In [ ]:
#Read the data or a subset thereof from disk, and save it in a pre-allocated output array A.
A =zeros(2048,2048); #pre-allocate matrix A
read!(laserData[1], A); #data

In [ ]:
close(laserData);

In [ ]:
#Constantes
ainit = [300. , 400. , 500.];
fwhminit = [2.0, 2.0 , 2.0];
laser =  LaserModel(λlaser,ainit,fwhminit);
Cinit = zeros(2,laser.nλ);

#BoundingBox
largeur = 4;
hauteur = 40;
dxmin = round(Int, largeur/2);
dxmax = round(Int, largeur/2);
dymin = 21#round(Int, hauteur/2);
dymax = 13#round(Int, hauteur/2);
mcx1 = median(cx1);
mcy1 = median(cy1);
mcx2 = median(cx2);
mcy2 = median(cy2);

N = length(cx0) #Nombre de lenslet (18908)

valid = (round.(cx0 .- dxmin).>0) .&  (round.(cx0 .+ dxmax).<2048) .&  
(round.(cy0 .- dymin).>0) .&  (round.(cy0 .+ dymax).<2048);

In [ ]:
#Initialiser tableaux de resultats avec possibilite d'avoir valeurs "missing"
atab = Array{Union{Float64,Missing}}(missing,3,N); #zeros(3,N);
fwhmtab = Array{Union{Float64,Missing}}(missing,3,N); #zeros(3,N);
ctab = Array{Union{Float64,Missing}}(missing,2,3,N); #zeros(2,3,N);
lmodtab = Array{Union{LensletModel,Missing}}(missing,N); #Array{LensletModel}(undef,N);

In [ ]:
atab

### Lenslet aleatoire sans prise en compte des pixels defectueux
xmin = cx0[i,1]-largeur/2 lenslet, ymin = cy0[i,1]-hauteur/2 lenslet

xmax = cx0[i,1]+largeur/2 lenslet, ymax = cy0[i,1]+hauteur/2 lenslet

Remarque: centre de l'image 0 = 1025x1025 pixel, donc il faut ajouter 1025 a xmin et xmax, ymin et ymax 

In [ ]:
i = 5
#Initialisation 
cx0i = cx0[i,1] .+1025; #centre de la lenslet
cy0i = cy0[i,1] .+1025;

xmin=cx0i -dxmin;
xmax=cx0i +dxmax;
ymin=cy0i -dymin;
ymax=cy0i +dymax; 

bbox = round(Int, BoundingBox(xmin, xmax, ymin, ymax)); 

lmodtab[i] = LensletModel(λ0,laser.nλ-1, bbox); #contient le model pour chaque lenslet

Cinit = zeros(2,laser.nλ);
Cinit[1:2,1:3] = [ [cx0i mcx1 mcx2]; [cy0i mcy1 mcy2] ];

#Fit du polynome pour chaque lenslet: optimisation
ldata = view(A, bbox);
lkl = LikelihoodIFS(lmodtab[i], laser, ldata);
cost(x::Vector{Float64}) = lkl(x)::Float64;
xinit = vcat([ainit[:],fwhminit[:],Cinit[:]]...);
xopt = vmlmb(cost, xinit; verb=50, maxeval=500);

#Sauvegarde des valeurs des parametres du modele dans des vecteurs
#ce qu'on veut garder: lmod sous forme de vecteur, 1 modele de lenslet par lenslet
(aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
atab[:,i] = aopt;
fwhmtab[:,i] = fwhmopt;
ctab[:,:,i]  = copt;


In [ ]:
#Affichage
x_axis = range(bbox.xmin, bbox.xmax, length = bbox.xmax-bbox.xmin+1);
y_axis = range(bbox.ymin, bbox.ymax, length = bbox.ymax-bbox.ymin+1);

In [ ]:
plt = scatter([cx0i], [cy0i], color=["gold"], label = "centre Anthony");
heatmap!(plt,x_axis, y_axis,  transpose(ldata), title = "lenslet 5", 
    xlabel = "x", ylabel = "y", xrange = [xmin-10,xmax+10], aspect_ratio = 1, legend = :bottomright)

In [ ]:
#Point de depart de l'optimiseur
UpdateDispModel(lmodtab[i].dmodel, Cinit);
UpdateLaserModel(laser,ainit,fwhminit);
limageinit = LensletLaserImage(lmodtab[i],laser)
plt = scatter([cx0i], [cy0i], color=["red"], label = "Initialisation");
heatmap!(plt, x_axis, y_axis,  transpose(limageinit), title = "Initialisation", 
    xlabel = "x", ylabel = "y", xrange = [xmin-10,xmax+10], aspect_ratio = 1, legend = :bottomright)

In [ ]:
#Point d'arrivee de l'optimiseur
UpdateDispModel(lmodtab[i].dmodel, copt);
UpdateLaserModel(laser,aopt,fwhmopt);
limageopt = LensletLaserImage(lmodtab[i],laser)
plt = scatter( [cx0i;copt[1,1]], [cy0i;copt[2,1]], color = ["red" , "blue"], 
    lab = ["initialisation" "optimisation"]);
heatmap!(plt, x_axis, y_axis,  transpose(limageopt), title = "Point d'arrivee de l'optimiseur", 
    xlabel = "x", ylabel = "y", xrange = [xmin-10,xmax+10], aspect_ratio = 1, legend = :bottomright)

In [ ]:
#Residus
heatmap(x_axis, y_axis, transpose(ldata .- limageopt), title = "Residus sans precision", 
    xlabel = "x", ylabel = "y", xrange = [xmin-10,xmax+10], aspect_ratio = 1 )

In [ ]:
@show(ainit,aopt,);

@show(fwhminit,fwhmopt);

@show(Cinit,copt);

### Lenslet aleatoire avec prise en compte des pixels defectueux

In [ ]:
badpix = Float64.(read(FITS("/home/user/stage/HR_4796-HD_95086/IFS_BP_corrected.fits")[1]));

In [ ]:
i = 5
#Initialisation 
cx0i = cx0[i,1] .+1025; #centre de la lenslet
cy0i = cy0[i,1] .+1025;

xmin=cx0i .-dxmin;
xmax=cx0i .+dxmax;
ymin=cy0i .-dymin;
ymax=cy0i .+dymax; 

bbox = round(Int, BoundingBox(xmin, xmax, ymin, ymax)); 

lmodtab[i] = LensletModel(λ0,laser.nλ-1, bbox); #contient le model pour chaque lenslet

Cinit = zeros(2,laser.nλ);
Cinit[1:2,1:3] = [ [cx0i mcx1 mcx2]; [cy0i mcy1 mcy2] ];



#Fit du polynome pour chaque lenslet: optimisation
ldata = view(A, bbox);
badpixview = view(badpix,bbox)
lkl = LikelihoodIFS(lmodtab[i], laser, ldata, badpixview);
cost(x::Vector{Float64}) = lkl(x)::Float64;
xinit = vcat([ainit[:],fwhminit[:],Cinit[:]]...);
xopt = vmlmb(cost, xinit; verb=50, maxeval=500);

#Sauvegarde des valeurs des parametres du modele dans des vecteurs
#ce qu'on veut garder: lmod sous forme de vecteur, 1 modele de lenslet par lenslet
(aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
atab[:,i] = aopt;
fwhmtab[:,i] = fwhmopt;
ctab[:,:,i]  = copt;

In [ ]:
#Affichage
x_axis = range(bbox.xmin, bbox.xmax, length = bbox.xmax-bbox.xmin+1);
y_axis = range(bbox.ymin, bbox.ymax, length = bbox.ymax-bbox.ymin+1);

In [ ]:
plt = scatter([cx0i], [cy0i], color=["gold"], label = "centre Anthony");
heatmap!(plt,x_axis, y_axis,  transpose(ldata), title = "lenslet 5", 
    xlabel = "x", ylabel = "y", xrange = [xmin-10,xmax+10], aspect_ratio = 1,  legend = :bottomright)

In [ ]:
#Point de depart de l'optimiseur
UpdateDispModel(lmodtab[i].dmodel, Cinit);
UpdateLaserModel(laser,ainit,fwhminit);
limageinit = LensletLaserImage(lmodtab[i],laser)
plt = scatter([cx0i], [cy0i], color=["red"], label = "Initialisation");
heatmap!(plt, x_axis, y_axis, transpose(limageinit), title = "Initialisation", xrange = [xmin-10,xmax+10], aspect_ratio = 1, legend = :bottomright)

In [ ]:
#Point d'arrivee de l'optimiseur
UpdateDispModel(lmodtab[i].dmodel, copt);
UpdateLaserModel(laser,aopt,fwhmopt);
limageopt = LensletLaserImage(lmodtab[i],laser)
plt = scatter( [cx0i;copt[1,1]], [cy0i;copt[2,1]], color = ["red" , "blue"], lab = ["initialisation" "optimisation"]);
heatmap!(plt, x_axis, y_axis, transpose(limageopt), title = "Point d'arrivee de l'optimiseur avec precision", xrange = [xmin-10,xmax+10], aspect_ratio = 1, legend = :bottomright)

In [ ]:
#Residus
heatmap(x_axis, y_axis, transpose(ldata .- limageopt), title = "Residus avec precision", xlabel = "x", ylabel = "y", xrange = [xmin-10,xmax+10], aspect_ratio = 1)

In [ ]:
@show(ainit,aopt,);

@show(fwhminit,fwhmopt);

@show(Cinit,copt);

### Ensemble des lenslet

In [ ]:
collect(1:N)[valid] #indice des lenslet valides

In [ ]:
@time for i in collect(1:N)[valid]
    #Initialisation
    cx0i = cx0[i,1] .+1025; #centre de la lenslet
    cy0i = cy0[i,1] .+1025;

    xmin=cx0i-dxmin;
    xmax=cx0i+dxmax;
    ymin=cy0i-dymin;
    ymax=cy0i+dymax; 
    if ymax>2048
        continue
    end 
    bbox = round(Int, BoundingBox(xmin, xmax, ymin, ymax));
    
    lmodtab[i] = LensletModel(λ0,laser.nλ-1, bbox); #contient le model pour chaque lenslet
    
    Cinit = [ [cx0i mcx1 mcx2]; [cy0i mcy1 mcy2] ]; #[1:2,1:3]
    
    #Fit du polynome pour chaque lenslet: optimisation
    ldata = view(A, bbox);    #lenslet laser data
    #badpixview = view(badpix,bbox)
    lkl = LikelihoodIFS(lmodtab[i], laser, ldata);#rajouter carte BP: badpixview
    cost(x::Vector{Float64}) = lkl(x);
    xinit = vcat([ainit[:],fwhminit[:],Cinit[:]]...);
    
    try #The try/catch statement allows for Exceptions to be tested for. much slower than using conditional branching
        xopt = vmlmb(cost, xinit; verb=false, maxeval=100);

        #Sauvegarde des valeurs des parametres du modele dans des vecteurs
        #ce qu'on veut garder: lmod sous forme de vecteur, 1 modele de lenslet par lenslet
        (aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
        atab[:,i] = aopt
        fwhmtab[:,i] = fwhmopt
        ctab[:,:,i]  = copt
    catch       #the symbol following catch will always be interpreted as a name for the exception
        continue
    end
        
    print(i)
end


In [ ]:
atab2 = collect(skipmissing(atab))
amp = FITS("amplitude.fits", "w");
write(amp, atab2)  # Write a new image extension with the data
close(amp)

In [ ]:
fwhmtab2 = collect(skipmissing(fwhmtab))
fwhm = FITS("fwhm.fits", "w");
write(fwhm, fwhmtab2)  # Write a new image extension with the data
close(fwhm)

In [ ]:
ctab2 = collect(skipmissing(ctab))
coeff = FITS("coeff.fits", "w");
write(coeff, ctab2)  # Write a new image extension with the data
close(coeff)

In [ ]:
#Affichage
x_axis = range(bbox.xmin, bbox.xmax, length = bbox.xmax-bbox.xmin+1);
y_axis = range(bbox.ymin, bbox.ymax, length = bbox.ymax-bbox.ymin+1);

In [ ]:
plt = scatter([cx0i], [cy0i], color=["gold"], label = "centre Anthony");
heatmap!(plt,x_axis, y_axis,  transpose(ldata), xlabel = "x", ylabel = "y", xrange = [xmin-10, xmax+10], aspect_ratio = 1, legend = :bottomright)

In [ ]:
#Point de depart de l'optimiseur
UpdateDispModel(lmodtab[i].dmodel, Cinit);
UpdateLaserModel(laser,ainit,fwhminit);
limageinit = LensletLaserImage(lmodtab[i],laser)
plt = scatter([cx0i], [cy0i], color=["red"], label = "Initialisation");
heatmap!(plt, x_axis, y_axis,  transpose(limageinit), title = "Initialisation", xlabel = "x", ylabel = "y", xrange = [xmin-10,xmax+10], aspect_ratio = 1, legend = :bottomright)

In [ ]:
#Point d'arrivee de l'optimiseur
UpdateDispModel(lmodtab[i].dmodel, copt);
UpdateLaserModel(laser,aopt,fwhmopt);
limageopt = LensletLaserImage(lmodtab[i],laser)
plt = scatter( [cx0i;copt[1,1]], [cy0i;copt[2,1]], color = ["red" , "blue"], lab = ["initialisation" "optimisation"]);
heatmap!(plt, x_axis, y_axis,  transpose(limageopt), title = "Point d'arrivee de l'optimiseur", xlabel = "x", ylabel = "y", xrange = [xmin-20,xmax+20], aspect_ratio = 1, legend = :bottomright)

In [ ]:
#Residus
heatmap(x_axis, y_axis, transpose(ldata .- limageopt), title = "Residus sans precision", xlabel = "x", ylabel = "y", xrange = [xmin-10,xmax+10], aspect_ratio = 1)

## 3 affichage

### Test avec donnees lamp
Permet de verifier si la bbox es convenable (si spectre est complet ou decoupee ou pas sur quelques lenslet)

In [ ]:
lamp = FITS("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/IFS_sim_spec.fits", "r")
B = zeros(2048,2048);
data = read!(lamp[1], B);

In [ ]:
i = 5

xmin=(cx0[i,1] .-dxmin) .+1026;
xmax=(cx0[i,1] .+dxmax) .+1026;
ymin=(cy0[i,1] .-dymin) .+1024;
ymax=(cy0[i,1] .+dymax) .+1024;

bbox = round(Int, BoundingBox(xmin, xmax, ymin, ymax));

x_axis = range(bbox.xmin, bbox.xmax, length = bbox.xmax-bbox.xmin+1);
y_axis = range(bbox.ymin, bbox.ymax, length = bbox.ymax-bbox.ymin+1);

plt = scatter(cx0[1:10,1], cx0[1:10,1])
ldata = view(B,bbox);
heatmap(y_axis, x_axis, ldata, title = "Exemple du spectre d'une lenslet au centre")


In [ ]:
#toute l'image
bbox = BoundingBox(1, 2048, 1, 2048);
x_axis = range(bbox.xmin, bbox.xmax, length = bbox.xmax-bbox.xmin+1);
y_axis = range(bbox.ymin, bbox.ymax, length = bbox.ymax-bbox.ymin+1);
plt = scatter(cx0[:,1], cy0[:,1])
heatmap!(plt,x_axis, y_axis,  transpose(data), title = "IFS_sim_spec.fits")

In [ ]:
f6 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_spec_corrected.fits", "r"); #donnees lamp
B =zeros(2048,2048); #pre-allocate matrix A  Array{Float64}(undef,2048,2048)
read!(f6[1], B);
bbox = BoundingBox(xmin=1013, xmax=1017, ymin=1050, ymax=1088);
ldata = view(B,bbox); # ex lenslet au centre de l'image [1012:1018, 1048:1092]
p1 = heatmap(ldata, title = "Exemple du spectre d'une lenslet au centre")


f7 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_spec_corrected.fits", "r"); #donnees lamp
C =zeros(2048,2048); #pre-allocate matrix A  Array{Float64}(undef,2048,2048)
read!(f7[1], C);
bbox = BoundingBox(xmin=214, xmax=218, ymin=272, ymax=311);
ldata = view(C,bbox); 
p2 = heatmap(ldata, title = "Exemple 1 du spectre d'une lenslet en peripherie")


f8 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_spec_corrected.fits", "r"); #donnees lamp
D =zeros(2048,2048); #pre-allocate matrix A  Array{Float64}(undef,2048,2048)
read!(f7[1], D);
bbox = BoundingBox(xmin=1501, xmax=1505, ymin=1574, ymax=1613);
ldata = view(D,bbox); 
p3 = heatmap(ldata, title = "Exemple 2 du spectre d'une lenslet en peripherie")


plot(p1, p2, p3, legend = false, layout = (3, 1), aspect_ratio = 1)

On voit que le spectre n'est pas decoupe.

In [ ]:
#test
plt = scatter(cx0[1:10,1] .+200, cy0[1:10,1] .+400)
heatmap!(plt,x_axis, y_axis,  transpose(data), title = "IFS_sim_spec.fits")

Methode 1: tracer des histogrammes de tous les coeff, visualisation des outliers. Ajustement d'une gaussienne, calcul de la moyenne de la variance, ce qui est superieur a 3σ est sorti.

Methode 2: test statistique par rapport au guess si methode 1 marche pas. Histogramme des valeurs de χ2

In [ ]:
h5 = histogram(atab)
h6 = histogram(fwhmtab)
h7 = histogram(ctab)
plot(h5, h6, h7, legend = false)

In [ ]:
#Statistics
@show std(aopt);
@show std(fwhmopt);
@show std(copt);

In [ ]:
@show var(aopt);
@show var(fwhmopt);
@show var(copt);

In [ ]:
lmod.dmodel.(λlaser)
heatmap(clamp.(ldata,0,1500), title = "Fit laser")

In [ ]:
for i in 14957:14970
    #Initialisation
    cx0i = cx0[i,1] .+1025;
    cy0i = cy0[i,1] .+1025;

    xmin=cx0i-dxmin;
    xmax=cx0i+dxmax;
    ymin=cy0i-dymin;
    ymax=cy0i+dymax; 
    if ymax>2048
        continue
    end  
    bbox = round(Int, BoundingBox(xmin, xmax, ymin, ymax));
    
    lmodtab[i] = LensletModel(λ0,laser.nλ-1, bbox); #contient le model pour chaque lenslet
     
    Cinit = [ [cx0i mcx1 mcx2]; [cy0i mcy1 mcy2] ];
    
    #Fit du polynome pour chaque lenslet: optimisation
    ldata = view(A, bbox);    #lenslet laser data
    lkl = LikelihoodIFS(lmodtab[i], laser, ldata);#rajouter carte BP
    cost(x::Vector{Float64}) = lkl(x);
    xinit = vcat([ainit[:],fwhminit[:],Cinit[:]]...);
    try
        xopt = vmlmb(cost, xinit; verb=false, maxeval=500); #verb=50

        #Sauvegarde des valeurs des parametres du modele dans des vecteurs
        (aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
        atab[:,i] = aopt
        fwhmtab[:,i] = fwhmopt
        ctab[:,:,i]  = copt
    catch
        continue
    end
    print(i)
end
